## Imports

In [1]:
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

import clustering

## Importando resultados

In [2]:
fac_result = clustering.import_best_result("data/melhor_resultado_fac.pickle")
fou_result = clustering.import_best_result("data/melhor_resultado_fou.pickle")
kar_result = clustering.import_best_result("data/melhor_resultado_kar.pickle")
todas_result = clustering.import_best_result("data/melhor_resultado_todas.pickle")


fac_fuzzy = clustering.import_fuzzy_partitions_from_csv("data/fuzzy_partitions_fac.csv")
fou_fuzzy = clustering.import_fuzzy_partitions_from_csv("data/fuzzy_partitions_fou.csv")
kar_fuzzy = clustering.import_fuzzy_partitions_from_csv("data/fuzzy_partitions_kar.csv")
todas_fuzzy = clustering.import_fuzzy_partitions_from_csv("data/fuzzy_partitions_todas.csv")

In [3]:
todas_result["times"]

9

## Melhores resultados

In [4]:
print("Melhor custo pro fac", fac_result["cost"])
print("Melhor custo pro fou", fou_result["cost"])
print("Melhor custo pro kar", kar_result["cost"])
print("Melhor custo com as três matrizes", todas_result["cost"])

Melhor custo pro fac 3383.2055017732046
Melhor custo pro fou 1800.4341935576729
Melhor custo pro kar 1517.5559412009745
Melhor custo com as três matrizes 18990.729309877664


### Modified partition coefficient

In [5]:
print('>> FAC (VIEW1)', clustering.calc_modified_partition_coefficient(fac_result["membership_degree"]))
print('>> FOU (VIEW2)', clustering.calc_modified_partition_coefficient(fou_result["membership_degree"]))
print('>> KAR (VIEW3)', clustering.calc_modified_partition_coefficient(kar_result["membership_degree"]))
print('>> TODAS (VIEW0)', clustering.calc_modified_partition_coefficient(todas_result["membership_degree"]))

>> FAC (VIEW1) 0.012656027846346563
>> FOU (VIEW2) 0.0003644924738648303
>> KAR (VIEW3) 0.0009918034380510532
>> TODAS (VIEW0) 0.5171485684755281


### Entropy

In [6]:
print('>> FAC (VIEW1)', clustering.calc_partition_entropy(fac_result["membership_degree"]))
print('>> FOU (VIEW2)', clustering.calc_partition_entropy(fou_result["membership_degree"]))
print('>> KAR (VIEW3)', clustering.calc_partition_entropy(kar_result["membership_degree"]))
print('>> TODAS (VIEW0)', clustering.calc_partition_entropy(todas_result["membership_degree"]))

>> FAC (VIEW1) 0.9773769238834532
>> FOU (VIEW2) 0.9993079325937968
>> KAR (VIEW3) 0.9981081541310196
>> TODAS (VIEW0) 0.44597844611709286


### Partições crisp

In [7]:
from itertools import chain

classses = clustering.get_instances_class(10)

### Adjusted Rand Index

In [8]:
from sklearn.metrics import adjusted_rand_score

fac_crisp, fac_classes = clustering.get_hard_patitions(fac_result["membership_degree"])
fou_crisp, fou_classes = clustering.get_hard_patitions(fou_result["membership_degree"])
kar_crisp, kar_classes = clustering.get_hard_patitions(fou_result["membership_degree"])
todas_crisp, todas_classes = clustering.get_hard_patitions(todas_result["membership_degree"])

print('>> FAC (VIEW1)', adjusted_rand_score(fac_classes, classses))
print('>> FOU (VIEW2)', adjusted_rand_score(fou_classes, classses))
print('>> KAR (VIEW3)', adjusted_rand_score(kar_classes, classses))
print('>> TODAS (VIEW0)', adjusted_rand_score(todas_classes, classses))

>> FAC (VIEW1) 0.28955002143740005
>> FOU (VIEW2) 0.0993059031935507
>> KAR (VIEW3) 0.0993059031935507
>> TODAS (VIEW0) 0.648792424321116


### F-measure

In [9]:
print('>> FAC (VIEW1)', f1_score(fac_classes, classses, average="micro"))
print('>> FOU (VIEW2)', f1_score(fou_classes, classses, average="micro"))
print('>> KAR (VIEW3)', f1_score(kar_classes, classses, average="micro"))
print('>> TODAS (VIEW0)', f1_score(todas_classes, classses, average="micro"))

>> FAC (VIEW1) 0.152
>> FOU (VIEW2) 0.04900000000000001
>> KAR (VIEW3) 0.04900000000000001
>> TODAS (VIEW0) 0.0195


### Erro de classificação

In [10]:
print('>> FAC (VIEW1)', 1 - accuracy_score(fac_classes, classses))
print('>> FOU (VIEW2)', 1 - accuracy_score(fou_classes, classses))
print('>> KAR (VIEW3)', 1 - accuracy_score(kar_classes, classses))
print('>> TODAS (VIEW0)', 1 - accuracy_score(todas_classes, classses))

>> FAC (VIEW1) 0.848
>> FOU (VIEW2) 0.951
>> KAR (VIEW3) 0.951
>> TODAS (VIEW0) 0.9805


## Melhor resultado

In [11]:
fac_result["name"] = "fac"
fou_result["name"] = "fou"
kar_result["name"] = "kar" 
todas_result["name"] = "todas"

melhor_resultado = min(fac_result, fou_result, kar_result, todas_result, key=lambda d:d["cost"])
print("Melhore resultado: ", melhor_resultado["name"])

Melhore resultado:  kar


### Prototipos

In [12]:
{i:p for i,p in enumerate(melhor_resultado["prototypes"])}

{0: [25, 78],
 1: [25, 78],
 2: [25, 78],
 3: [25, 78],
 4: [25, 78],
 5: [25, 78],
 6: [1890, 78],
 7: [25, 78],
 8: [1494, 1890],
 9: [25, 78]}

### Parições crisp

In [13]:
membership_degree = melhor_resultado["membership_degree"]
melhor_hard, melhor_classes = clustering.get_hard_patitions(membership_degree)

for number, group in enumerate(melhor_hard):
    print("Grupo: ", number, "Elementos: ", group, "Quantidade: ", len(group))

print("Total de elementos somados:",  sum([len(g) for g in melhor_hard]))

Grupo:  0 Elementos:  [] Quantidade:  0
Grupo:  1 Elementos:  [] Quantidade:  0
Grupo:  2 Elementos:  [] Quantidade:  0
Grupo:  3 Elementos:  [] Quantidade:  0
Grupo:  4 Elementos:  [] Quantidade:  0
Grupo:  5 Elementos:  [] Quantidade:  0
Grupo:  6 Elementos:  [ 135  207  220  227  228  238  239  240  249  252  253  254  265  271
  276  291  302  317  321  325  330  331  332  333  334  338  343  350
  352  357  361  363  364  367  369  381  387  389  390  395  398  403
  479  529  559  563  593  618  654  707  713  724  727  733  744  769
  789  795  808  811  814  824  830  837  850  856  859  864  868  876
  877  881  882  883  886  887  891  895  897  908  911  912  920  922
  924  926  929  930  934  945  951  955  956  961  963  966  976  977
  982  988  990  997  998  999 1002 1006 1013 1020 1021 1023 1026 1027
 1033 1035 1048 1050 1051 1069 1075 1076 1078 1079 1089 1090 1096 1102
 1106 1122 1123 1124 1126 1130 1132 1136 1138 1140 1145 1150 1152 1155
 1159 1161 1166 1170 1190 11

###  Modified partition coefficient

In [14]:
print('>> ', clustering.calc_modified_partition_coefficient(melhor_resultado["membership_degree"]))

>>  0.0009918034380510532


### Adjusted Rand Index

In [15]:
print('>> ', adjusted_rand_score(fac_classes, classses))

>>  0.28955002143740005


### F-measure

In [16]:
print('>> ', f1_score(melhor_classes, classses, average="micro"))

>>  0.017


### Erro de classificação

In [17]:
print('>> ', 1 - accuracy_score(melhor_classes, classses))

>>  0.983
